In [2]:
import json
import numpy as np
import random

In [3]:
#dict of items vs weight
Items_w={'ID=0':2.94,'ID=1':2.12,'ID=2':2.26,'ID=3':1.39,'ID=4':0.8,'ID=5':0.52,'ID=6':0.68,'ID=7':2.27,'ID=8':2.18,'ID=9':2.69,'ID=10':0.58,'ID=11':2.79,'ID=12':1.08,'ID=13':2.4,'ID=14':1.89,'ID=15':2.64,
      'ID=16':2.21,'ID=17':0.77,'ID=18':2.27,'ID=19':2.01,'ID=20':2.67,'ID=21':2.44,'ID=22':1.85,'ID=23':1.22}

In [4]:
#set of items
I={'ID=0','ID=1','ID=2','ID=3','ID=4','ID=5','ID=6','ID=7','ID=8','ID=9','ID=10','ID=11','ID=12','ID=13','ID=14','ID=15',
      'ID=16','ID=17','ID=18','ID=19','ID=20','ID=21','ID=22','ID=23'}

In [5]:
#Nested dict
#Shelves with ID of items, weight and quantities
S= {'0':{'ID=10': [0.58, 100]}, '1':{'ID=6': [0.68, 100]}, '2': {'ID=5': [0.52, 100]}, '3': {'ID=11': [2.79, 100]}, '4': {'ID=8': [2.18, 100]}, '5': {'ID=18': [2.27, 100]}, '6': {'ID=15': [2.64, 100]},
  '7': {'ID=0': [2.94, 100]}, '8': {'ID=13': [2.4, 100]}, '9': {'ID=7': [2.27, 100]}, '10': {'ID=3': [1.39, 100]}, '11': {'ID=4': [0.8, 100]}, '12': {'ID=21': [2.44, 100]}, '13': {'ID=16': [2.21, 100]},
  '14':{'ID=22': [1.85, 100]}, '15': {'ID=1': [2.12, 100]}, '16': {'ID=2': [2.26, 100]}, '17': {'ID=20': [2.67, 100]}, '18': {'ID=12': [1.08, 100]}, '19': {'ID=17': [0.77, 100]}, '20': {'ID=23': [1.22, 100]}, 
   '21': {'ID=9': [2.69, 100]}, '22': {'ID=19': [2.01, 100]}, '23': {'ID=14': [1.89, 100]}}

In [6]:
#Orders with items and quantities: nested dict
O={'0':{'ID=3':1,'ID=7':1}, '1':{'ID=7':1,'ID=6':2, 'ID=8':1, 'ID=0':2, 'ID=2':1}, '2':{'ID=7':1,'ID=5':1}, '3':{'ID=7':2,'ID=1':1, 'ID=5':1, 'ID=6':1},
   '4':{'ID=3':2,'ID=5':1, 'ID=4':1}, '5':{'ID=5':1,'ID=2':3, 'ID=4':2, 'ID=7':1, 'ID=8':1}, '6':{'ID=0':3,'ID=3':1,'ID=1':1}, '7':{'ID=1':1,'ID=8':1,'ID=7':1},
  '8':{'ID=6':1,'ID=0':2,'ID=2':2,'ID=1':1,'ID=5':1,'ID=4':1 }, '9':{'ID=7':1,'ID=8':1,'ID=6':1,'ID=4':1,'ID=2':1,'ID=1':1 }}

#### Data: Distances among shelves and depots

In [7]:
# Opening JSON file
f = open('layout_sku_24_2.json',)
  
# returns JSON object as 
# a dictionary
dist = json.load(f)
  
# Iterating through the json
# list
#for i in data['emp_details']:
#    print(i)
  
# Closing file
f.close()

In [8]:
#list of Depots
D=['OutD0', 'OutD1']

## Prepare the inputs for the algoritm 3: Order to depots


### Function: V_p_SC set of shelves containing item p

In [9]:
#Function: V_p_SC set of shelves containing item p
#V_p_SC.append()
p='ID=7'
def get_shelves(p):
    V_p_SC=set()
    for shelf, itms in S.items():
        for key,value in itms.items():
            if p==key:
                #print(order)
                V_p_SC.add(shelf)
    return list(V_p_SC)
V_p_SC=get_shelves(p)    
V_p_SC

['9']

### Function get list of items from an order o


In [11]:
#get list of items of an order o
o='0'
def get_items(o):
    lst_items=[]
    for order, order_info in O.items():
        for key in order_info.keys():
            if order==o:
                lst_items.append(key)
    return lst_items
lst_items=get_items(o)
lst_items

['ID=3', 'ID=7']

### Function to get distance from an order to a depot: formula (30) in Xie paper

In [12]:
o='0' #order
#depot i
i=D[0]
def dist_oi(o,i):
    lst_items=get_items(o)
    d_oi=0
    for p in lst_items:    
        V_p_SC=get_shelves(p) #get set of shelves containing p
        dij=[]
        for j in V_p_SC:
            dij.append(dist[i][j])
        d_oi+=min(dij)
    return d_oi #already outputing correct results
dist_oi(o,i)

24.4

### Function: Create input D_o_hat: an array of (10,2) --> distances of 10 orders and 2 depots  and: \mu_o, for all orders o, is an array of (10,1)

In [13]:
def set_dist(O,D):
    D_o_hat=np.zeros([len(D),len(O)])
    for i in range(len(D)):
        for j in range(len(O)):
            D_o_hat[i,j]=dist_oi(list(O.keys())[j],D[i])
    #calculate D_o_hat for the case when D has more than 2 depots! here it has 2--> no need
    mu_o=np.abs(D_o_hat[0]-D_o_hat[1])
    return mu_o, D_o_hat

### function to sort O, D_o_hat according to mu_o in decending order


In [14]:
# sort O and D_o_hat according to mu_o in decending order
O={'0':{'ID=3':1,'ID=7':1}, '1':{'ID=7':1,'ID=6':2, 'ID=8':1, 'ID=0':2, 'ID=2':1}, '2':{'ID=7':1,'ID=5':1}, '3':{'ID=7':2,'ID=1':1, 'ID=5':1, 'ID=6':1},
   '4':{'ID=3':2,'ID=5':1, 'ID=4':1}, '5':{'ID=5':1,'ID=2':3, 'ID=4':2, 'ID=7':1, 'ID=8':1}, '6':{'ID=0':3,'ID=3':1,'ID=1':1}, '7':{'ID=1':1,'ID=8':1,'ID=7':1},
  '8':{'ID=6':1,'ID=0':2,'ID=2':2,'ID=1':1,'ID=5':1,'ID=4':1 }, '9':{'ID=7':1,'ID=8':1,'ID=6':1,'ID=4':1,'ID=2':1,'ID=1':1 }}
mu_o=np.array([3.6, 0.9, 0. , 1.8, 6.3, 9.9, 0.9, 4.5, 1.8, 9. ])
D_o_hat=np.array([[24.4 , 58.25, 22.6 , 45.6 , 37.05, 62.75, 35.65, 37.45, 71.3 ,
        74.9 ],
       [20.8 , 57.35, 22.6 , 47.4 , 30.75, 52.85, 34.75, 32.95, 69.5 ,
        65.9 ]])
def sortOD_mu(mu_o,D_o_hat,O):
    #get the indexes of mu_o in decending order
    mu_o_sorted=list(np.argsort(mu_o)[::-1])
    O_sorted_lst=[]
    D_o_hat_sorted=[[],[]]
    for k in mu_o_sorted:
        O_sorted_lst.append(list(O.items())[k])
        D_o_hat_sorted= np.append(D_o_hat_sorted, D_o_hat[:,k].reshape(2,1), axis=1)
    keys=[]
    values=[]
    for (key,value) in O_sorted_lst:
        keys.append(key)
        values.append(value)   
    O_sorted=dict(zip(keys, values))
    return O_sorted, D_o_hat_sorted   

O_sorted, D_o_hat_sorted=sortOD_mu(mu_o,D_o_hat,O)
O_sorted
#D_o_hat_sorted

{'5': {'ID=5': 1, 'ID=2': 3, 'ID=4': 2, 'ID=7': 1, 'ID=8': 1},
 '9': {'ID=7': 1, 'ID=8': 1, 'ID=6': 1, 'ID=4': 1, 'ID=2': 1, 'ID=1': 1},
 '4': {'ID=3': 2, 'ID=5': 1, 'ID=4': 1},
 '7': {'ID=1': 1, 'ID=8': 1, 'ID=7': 1},
 '0': {'ID=3': 1, 'ID=7': 1},
 '8': {'ID=6': 1, 'ID=0': 2, 'ID=2': 2, 'ID=1': 1, 'ID=5': 1, 'ID=4': 1},
 '3': {'ID=7': 2, 'ID=1': 1, 'ID=5': 1, 'ID=6': 1},
 '6': {'ID=0': 3, 'ID=3': 1, 'ID=1': 1},
 '1': {'ID=7': 1, 'ID=6': 2, 'ID=8': 1, 'ID=0': 2, 'ID=2': 1},
 '2': {'ID=7': 1, 'ID=5': 1}}

In [15]:
#dict of items vs weight
Items_w={'ID=0':2.94,'ID=1':2.12,'ID=2':2.26,'ID=3':1.39,'ID=4':0.8,'ID=5':0.52,'ID=6':0.68,'ID=7':2.27,'ID=8':2.18,'ID=9':2.69,'ID=10':0.58,'ID=11':2.79,'ID=12':1.08,'ID=13':2.4,'ID=14':1.89,'ID=15':2.64,
      'ID=16':2.21,'ID=17':0.77,'ID=18':2.27,'ID=19':2.01,'ID=20':2.67,'ID=21':2.44,'ID=22':1.85,'ID=23':1.22}
#o={'ID=7': 2, 'ID=1': 1, 'ID=5': 1, 'ID=6': 1}
o='6'
def weight_order(o):
    o_info=O[o]
    w_o=0
    for key,value in o_info.items():
        w_o+= Items_w.get(key)*value
    return w_o
weight_order(o)

12.330000000000002

## Algorithm 3: Greedy assignment of orders to depots


In [16]:
#Function similarity(O_temp, O_check)
# O_temp=[0, 7]-->items 3,7, 1,8
#O_check=[1,2,3]
#Output=[([orders,number of the mutual items with Otemp])]
#------------------------
#O_temp=[0], O_check=[1,2,3,4*,8]--> output=[([4*],2),([1,2,3],1),([8],0)]; '4*':{'ID=7':2,'ID=3':2,'ID=5':1, 'ID=4':1}
#Combine minDist and Similarity
#minDist(O_temp,O_check)-->prefered list=[2,3,1,8,4*]
#similarity(O_temp,O_check)--> list_similarity=[([4*],2),([1,2,3],1),([8],0)]; '4*':{'ID=7':2,'ID=3':2,'ID=5':1, 'ID=4':1}
# loof over preferlist from the beginning, which one stands in the first list_similarity, then we will take it
#priotize the similarity more: Choose 4
#--------------------------------------
#minDist(O_temp,O_check)-->prefered list=[2,3,1,8]
#similarity(O_temp,O_check)--> list_similarity=[([1,2,3],1),([8],0)]; '4*':{'ID=7':2,'ID=3':2,'ID=5':1, 'ID=4':1}
# loof over preferlist from the beginning, which one stands in the first list_similarity, then we will take it
#priotize the similarity more: there are 1,2,3 with similarity=1, choose the one with the best minDist-->choose order 2


O={'0':{'ID=3':1,'ID=7':1}, '1':{'ID=7':1,'ID=6':2, 'ID=8':1, 'ID=0':2, 'ID=2':1}, '2':{'ID=7':1,'ID=5':1}, '3':{'ID=7':2,'ID=1':1, 'ID=5':1, 'ID=6':1},
   '4':{'ID=3':2,'ID=5':1, 'ID=4':1}, '5':{'ID=5':1,'ID=2':3, 'ID=4':2, 'ID=7':1, 'ID=8':1}, '6':{'ID=0':3,'ID=3':1,'ID=1':1}, '7':{'ID=1':1,'ID=8':1,'ID=7':1},
  '8':{'ID=6':1,'ID=0':2,'ID=2':2,'ID=1':1,'ID=5':1,'ID=4':1 }, '9':{'ID=7':1,'ID=8':1,'ID=6':1,'ID=4':1,'ID=2':1,'ID=1':1 }}

O_sorted={'5': {'ID=5': 1, 'ID=2': 3, 'ID=4': 2, 'ID=7': 1, 'ID=8': 1},
 '9': {'ID=7': 1, 'ID=8': 1, 'ID=6': 1, 'ID=4': 1, 'ID=2': 1, 'ID=1': 1},
 '4': {'ID=3': 2, 'ID=5': 1, 'ID=4': 1},
 '7': {'ID=1': 1, 'ID=8': 1, 'ID=7': 1},
 '0': {'ID=3': 1, 'ID=7': 1},
 '8': {'ID=6': 1, 'ID=0': 2, 'ID=2': 2, 'ID=1': 1, 'ID=5': 1, 'ID=4': 1},
 '3': {'ID=7': 2, 'ID=1': 1, 'ID=5': 1, 'ID=6': 1},
 '6': {'ID=0': 3, 'ID=3': 1, 'ID=1': 1},
 '1': {'ID=7': 1, 'ID=6': 2, 'ID=8': 1, 'ID=0': 2, 'ID=2': 1},
 '2': {'ID=7': 1, 'ID=5': 1}}

D_o_hat_sorted=np.array([[62.75, 74.9 , 37.05, 37.45, 24.4 , 45.6 , 71.3 , 35.65, 58.25,
        22.6 ],
       [52.85, 65.9 , 30.75, 32.95, 20.8 , 47.4 , 69.5 , 34.75, 57.35,
        22.6 ]])

D_o_hat=np.array([[24.4 , 58.25, 22.6 , 45.6 , 37.05, 62.75, 35.65, 37.45, 71.3 ,
        74.9 ],
       [20.8 , 57.35, 22.6 , 47.4 , 30.75, 52.85, 34.75, 32.95, 69.5 ,
        65.9 ]])
mu_o=np.array([3.6, 0.9, 0. , 1.8, 6.3, 9.9, 0.9, 4.5, 1.8, 9. ])
#capacity of cobot: 18kg
c=18 
#determine B_star = overal minimum number of batches required to carry out all orders: formula (32) in the Xie paper
#initialize total weight of the set of all orders O
W_O = 0
for o in O.keys():
    W_O += weight_order(o)
B_star=np.rint(W_O/c)+1
    
def orderToDepot(O, mu_o, D_o_hat):
    #Initialize W_total at depots outD0 and outD1 in an one-dimensional array
    W_total=[0,0]
    #save the assignment of orders to depots in a dictionary
    Woi={}
    #save the assignment of order to depots in an array
    w_oi=np.zeros([len(D),len(O)])  #2x10
    #weight of an order
    #total weight at depot i
    # sort O according to mu_0
    O_sorted, D_o_hat_sorted =sortOD_mu(mu_o,D_o_hat,O)
    k=0
    for o in O_sorted.keys():
        #weight of order o
        wo=weight_order(o)
        #assign o to depot i
        i=np.argmin(D_o_hat_sorted[:,0]) 
        not_i=np.argmax(D_o_hat_sorted[:,0]) 
        #check balancing, np.rint = round up
        if ((W_total[i]+wo)/c) <= np.rint(B_star/len(D)):
            w_oi[i,k]=1
            Woi[o]=D[i]
            W_total[i]+=wo
            #eliminate o from O_sorted
            #O_sorted.pop(o, None) # no need to delete in case there are only 2 depots
        else:
            #assign o to the other depot
            w_oi[not_i,k]=1
            Woi[o]=D[not_i]
            W_total[not_i]+=wo           
        #increase counting variable one unit more
        k+=1
    return w_oi, Woi

#Woi 
w_oi=orderToDepot(O, mu_o, D_o_hat)
w_oi     

(array([[0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]]),
 {'5': 'OutD1',
  '9': 'OutD1',
  '4': 'OutD1',
  '7': 'OutD1',
  '0': 'OutD1',
  '8': 'OutD1',
  '3': 'OutD0',
  '6': 'OutD0',
  '1': 'OutD0',
  '2': 'OutD0'})

## 4. Algorithm 4: Order Batching and routing
In this algorithm the order o of the depot i needed to be assigned to batches while forming tours(picking items with certain ordering but orders are not necessary to be picked one by one (check project description)) without violating the batch capacity (cobot capacity).

#### Input of the algorithm 4:

In [17]:
# O_asg with orders o_i assigned to depot i
O_asg={'5': 'OutD1',
  '9': 'OutD1',
  '4': 'OutD1',
  '7': 'OutD1',
  '0': 'OutD1',
  '8': 'OutD1',
  '3': 'OutD0',
  '6': 'OutD0',
  '1': 'OutD0',
  '2': 'OutD0'}
#O_0 with orders o assigned to depot 0
O_0=  {  '3': 'OutD0',
  '6': 'OutD0',
  '1': 'OutD0',
  '2': 'OutD0'}
#O_1 with orders o assigned to depot 1
O_1={'5': 'OutD1',
  '9': 'OutD1',
  '4': 'OutD1',
  '7': 'OutD1',
  '0': 'OutD1',
  '8': 'OutD1'}

### 4.1 Function minDist(O_check, O_temp)
the minimum possible distance from any item of o to any item of the orders in the set O_temp in the batch. If O_temp is empty, o is chosen randomly.
In doing so, we can decide whether we should include o to the batch.

O_check is set of the orders to be assigned in the batch
O_temp is the set of the orders that are already assigned in the batch 

##### function DistOO
to check the distance of an order to the other order: summing up all the possible distances of the items in order 1 to any item of the order 2
* In the single shelf policy: it is easy because then the distance btw 2 items is just the distance btw two shelves, here is that approach
* In the mixed shelf: a bit more problematic because an item could be stored in different shelves, and the distance btw 2 items could be the distance among many different shelves, we dont know which shelves we should consider: think about it later! 

In Xie paper, there is a sentence: 'As an item can be stored in
many shelves, the shelf that minimizes the distance to any of the shelves visited
in the current batch is determined.'

##### Remark:
As in this task 2.1, the main purpose is to initialize a solution (feasible), we dont have to care about the objective function. 
Using distance btw depots, shelves or items vs items to initialize a solution is also reasonable because time = distance/speed, 
and speed of cobots and human pickers are given.

In [18]:
#function to calculate distance btw 2 items, this can be used for mixed-shelf case
def distII(itm1, itm2):
    #get shelves containing item p
    V1=get_shelves(itm1)
    V2=get_shelves(itm2)
    dii=0
    for s1 in V1:
        for s2 in V2:
            dii+=dist[s1][s2]
    return dii
itm1='ID=3'
itm2='ID=7'
dii=distII(itm1,itm2)
dii

0.8999999999999986

In [19]:
#not yet needed
#function to calculate distance btw 2 orders: at the moment this function is not necessary
def distOO(o1, o2):
    lst1=get_items(o1)
    lst2=get_items(o2)
    d=0
    for itm1 in lst1:
        for itm2 in lst2:
            d+=distII(itm1, itm2)
    return d

In [20]:
#combine items of a list of orders 
#O_temp: list of orders
O_temp=['0','2']
def combine_itm(O_temp):
    set_comb=set()
    #set3 = set1.union(set2)
    for o in O_temp:
        set_comb=set_comb.union(set(get_items(o)))
    return list(set_comb)
lst_comb=combine_itm(O_temp)
lst_comb

['ID=3', 'ID=7', 'ID=5']

#### minDist that requires both Oi and O_temp non-empty


In [21]:
#Inputs:
#O_check= list of orders to choose the next one to be assigned to the batch
#O_temp: list of orders that are already assigned to the batch 
#Output: turn the permutation of orders in O_check with preferences (with distance to O_temp in ascending order, min stands in the first position):
#which order should choose first will stay first in the list
O_temp=['0']
O_check=['1','2','3']
#O_check=['1','3']
def minDist(O_check, O_temp):
    comb_temp=combine_itm(O_temp)#list
    lst=[]
    for o in O_check:
        # total distance
        d=0
        #get items of o
        o_itms=get_items(o) #list
        for itm1 in comb_temp:
            for itm2 in o_itms:
                d+=distII(itm1, itm2)
        lst.append(d)
    #arrange list of order with distances to O_temp in ascending
    ind_dec=list(np.argsort(lst))
    O_prefer=[]
    for i in ind_dec:
        O_prefer.append(O_check[i])
        
    return O_prefer
O_prefer=minDist(O_check, O_temp)
O_prefer

['2', '3', '1']

### Algorithm 5. Function greedyTour(tour=[t_1,...,t_n,t_1],o)
Update tour when adding 1 more order

In [22]:
# Check correct answer: j=3, 10-->2-->7, d_add=0, because d(10,7)=2,7; d(2,7)=0.9; d(10,2)=1.8
tour=['OutD0', '9','10','7','OutD0']
s='2'
def addMinShelf(tour,s):
    lst=[]
    for j in range(1,len(tour)):
        d_add_j=dist[tour[j-1]][s]+dist[s][tour[j]]-dist[tour[j-1]][tour[j]]
        lst.append(d_add_j)
    #print(lst)
    d_add=np.min(lst)
    j=np.argmin(lst)+1 # real j will be j+1
    return d_add, j
d_add, j = addMinShelf(tour,s)   
d_add, j

(0.0, 1)

##### Function greedyTour


In [23]:
#without duplicate shelves
#greedyTour version without duplicate, should update the quantities of items in each shelf
#update also in function batch to tour the quantities of items in each shelf


#inputs: tour=[t0,t1,...t_n, t0], where t0 is one of the depot. order o, to update the tour, 
#with the assumption that the batch is already feasible
#output: the updated tour

def greedyTour(tour,o):
    #initialize
    #d_add_best=1000000
    P_oC=get_items(o) # list of items in o
    #get items with quantities of items in o
    itms_quant_o=O[o]
    for p in P_oC:
        V_p=get_shelves(p)#list of shelves containing p
        for s in V_p:
            #skip 1 iteration if tour already contains s, this is to avoid duplicate shelf in tour
            if s in set(tour):
                continue
            #print('s=',s)
            d_add, j = addMinShelf(tour,s) 
            #print('d_add, j=',(d_add, j))
            tour.insert(j,s)
        #update the quantity of p in shelf s_best
        #S[s_best][p][1] = S[s_best][p][1]- itms_quant_o[p]
    return tour

In [24]:
#with duplicate shelves
#inputs: tour=[t0,t1,...t_n, t0], where t0 is one of the depot. order o, to update the tour, 
#with the assumption that the batch is already feasible
#output: the updated tour

def greedyTour2(tour,o):
    #initialize
    #d_add_best=1000000
    P_oC=get_items(o) # list of items in o
    #get items with quantities of items in o
    itms_quant_o=O[o]
    for p in P_oC:
        V_p=get_shelves(p)#list of shelves containing p
        for s in V_p:
            #skip 1 iteration if tour already contains s, this is to avoid duplicate shelf in tour
            if s in set(tour):
                #insert one more s right beside s
                tour.insert(tour.index(s),s)
            #print('s=',s)
            d_add, j = addMinShelf(tour,s) 
            #print('d_add, j=',(d_add, j))
            tour.insert(j,s)
        #update the quantity of p in shelf s_best
        #S[s_best][p][1] = S[s_best][p][1]- itms_quant_o[p]
    return tour

In [27]:
#the distance of a tour
def distTour(tour):
    d=0
    for i in range(len(tour)-1):
        d+=dist[tour[i]][tour[i+1]]
    return d
tour1=['OutD0', '11', '2', '15', '16', '7', '1', 'OutD0']

tour2=['OutD0', '11', '2',  '7', '1','15', '16', 'OutD0']
tour3=['OutD0', '15', '16', '11', '2',  '7', '1','OutD0']
print(distTour(tour1))
print(distTour(tour2))
print(distTour(tour3))

61.099999999999994
48.099999999999994
40.900000000000006


### Batch to tour
arrange a batch in tour

In [28]:
def batchToTour(Bi,tour):
    for o in Bi:
        tour=greedyTour(tour,o)
    return tour

### 4.3 Algorithm to define batches and tours

In [29]:
#depot 0
# to submit: should be looped over the depots
def orderToBatch_Tour2(Oi):
    Bi=[]
    batchnr=1
    #set of orders assigned to OutD0
    #Oi=O0
    #batch capacity or cobot capacity
    c=18
    while len(Oi)!=0:
        #Initialization for the new batch
        #random an order for O_temp
        O_temp=random.sample(Oi, 1) #O_temp=['6']
        o=O_temp[0]
        weight_O_temp= weight_order(o)
        print('weight_O_temp initialization=', weight_O_temp)
        tour=['OutD0','OutD0'] #[di, di]
        tour=greedyTour(tour,o)
        Oi=Oi.difference(set(o))
        lst_preferences=minDist(list(Oi), O_temp)
        print('lst_preferences=', lst_preferences)
        while (weight_O_temp<c):
            for o in lst_preferences:
                if (weight_O_temp + weight_order(o)) < c: 
                    weight_O_temp += weight_order(o)
                    tour=greedyTour(tour,o)
                    O_temp.append(o)
                    Oi=Oi.difference(set(o))
                    print('O_temp=',O_temp)
            Bi.append((O_temp,tour,batchnr))
            batchnr += 1
            break  # this is needed to exist the second while loop
    return Bi

### for arbitrary depot

In [32]:
#depot i
#list of depots:
#D=['OutD0','OutD1'], D[0]= 'OutD0', D[1]= 'OutD1'
#function: def orderToBatch_Tour(Oi,i)

def orderToBatch_Tour(Oi,i):
    #Oi is the set of orders assigned to depot i
    Bi=[]
    batchnr=1
    #set of orders assigned to OutD0
    #Oi=O0
    #batch capacity or cobot capacity
    c=18
    while len(Oi)!=0:
        #Initialization for the new batch
        #random an order for O_temp
        O_temp=random.sample(Oi, 1) #O_temp=['6']
        o=O_temp[0]
        weight_O_temp= weight_order(o)
        print('weight_O_temp initialization=', weight_O_temp)
        if i==0:
            tour=['OutD0','OutD0'] #[di, di]
        elif i==1:
            tour=['OutD1','OutD1']
        tour=greedyTour(tour,o)
        Oi=Oi.difference(set(o))
        lst_preferences=minDist(list(Oi), O_temp)
        print('lst_preferences=', lst_preferences)
        while (weight_O_temp<c):
            for o in lst_preferences:
                if (weight_O_temp + weight_order(o)) < c: 
                    weight_O_temp += weight_order(o)
                    tour=greedyTour(tour,o)
                    O_temp.append(o)
                    Oi=Oi.difference(set(o))
                    print('O_temp=',O_temp)
            Bi.append((O_temp,tour,batchnr))
            batchnr += 1
            break  # this is needed to exist the second while loop
    return Bi

In [33]:
# Data
#O_0 with orders o assigned to depot 0
O_0=  {'0': 'OutD0',
  '8': 'OutD0',
  '3': 'OutD0',
  '6': 'OutD0',
  '1': 'OutD0',
  '2': 'OutD0'}
#O_1 with orders o assigned to depot 1
O_1={'5': 'OutD1',
  '9': 'OutD1',
  '4': 'OutD1',
  '7': 'OutD1'}
#set of orders assigned to 0
O0={'0','8','3','6','1','2'}
O1={'5','9','4','7'}
orderToBatch_Tour(O0,0)

weight_O_temp initialization= 14.52
lst_preferences= ['0', '2', '6', '3', '1']
O_temp= ['8', '2']
weight_O_temp initialization= 12.330000000000002
lst_preferences= ['0', '3', '1']
O_temp= ['6', '0']
weight_O_temp initialization= 13.950000000000001
lst_preferences= ['3']
weight_O_temp initialization= 7.859999999999999
lst_preferences= []


[(['8', '2'], ['OutD0', '15', '16', '11', '9', '2', '7', '1', 'OutD0'], 1),
 (['6', '0'], ['OutD0', '15', '10', '9', '7', 'OutD0'], 2),
 (['1'], ['OutD0', '7', '1', '4', '9', '16', 'OutD0'], 3),
 (['3'], ['OutD0', '15', '9', '2', '1', 'OutD0'], 4)]

In [34]:
orderToBatch_Tour(O1,1)

weight_O_temp initialization= 13.349999999999998
lst_preferences= ['4', '7', '9']
O_temp= ['5', '4']
weight_O_temp initialization= 10.309999999999999
lst_preferences= ['7']
O_temp= ['9', '7']


[(['5', '4'], ['OutD1', '16', '11', '10', '4', '9', '2', 'OutD1'], 1),
 (['9', '7'], ['OutD1', '11', '4', '1', '9', '16', '15', 'OutD1'], 2)]

### Local Search


#### SA algorithm

In [35]:
#function Temperature temper
tour=['OutD0','3','1','19','13','22','21','OutD0']
#tour_prime=['OutD0','1','3','21','13','9','22','OutD0']
def temper(tour):
    lst=[]
    for s1 in tour:
        for s2 in tour:
            if s1!=s2:
                lst.append(dist[s1][s2])
    t=max(lst)-min(lst)
    return t
    
    


In [36]:
def dist_tour(tour):
    s=0
    for i in range(len(tour)-1):
        s+=dist[tour[i]][tour[i+1]]
    return s
dist_tour(tour)

44.5

In [37]:
#function Prob(tour,tour',T)
def Prob(tour,tour_prime,T):
    if dist_tour(tour_prime) < dist_tour(tour):
        return 1
    elif dist_tour(tour_prime) == dist_tour(tour):
        return 0
    else:
        return np.exp(-(dist_tour(tour_prime) - dist_tour(tour))/T)

#### cooling coefficient alpha, slightly less than 1, let's take alpha=0.9
#### 2opt operator


In [39]:
def two_opt(tour,i,j):
    tour_prime=[]
    t1=tour[0:(i+1)]
    t2=tour[(i+1):j]
    t2.reverse()
    t3=tour[j::]
    tour_prime=t1+t2+t3
    return tour_prime
    

### Function SA

In [40]:
def SA(tour):
    k=0
    k_no_change=0
    T=temper(tour)
    alpha=0.9
    tour_orig=tour
    cond=True
    while (cond==True):
    #while ((k<2) or (T>0.1) or (k_no_change<50)):
        for i in range(len(tour_orig)-2):
            for j in range(i+2,len(tour_orig)-1):
                tour_prime=two_opt(tour_orig,i,j)
                if Prob(tour,tour_prime,T)> 0.5:
                    #print(tour_prime)
                    tour=tour_prime
                    #print(tour_prime,i,j)
                    k_no_change=0
                else:
                    k_no_change+=1
        T=alpha*T
        k+=1
        if ((k>100) or (T<0.1) or (k_no_change>50)):
            cond=False    
    return tour

### Exchange orders


In [43]:
#check constraint
def weight_batch(b):
    w=0
    for o in b:
        w+=weight_order(o)
    return w      
def weight_batches(b_ci):
    w_b=0
    for b in b_ci:
        w_b+=weight_batch(b)
    return w_b
        

In [44]:
def constraint_bat(batches):
    l=len(batches)
    k=0
    for b in batches:
        if weight_batch(b) <=18:
            k+=1
    if k==l:
        return True
    else:
        return False
    

In [45]:
def check_wrap(bat0,bat1,o0,o1):
    w0=weight_batch(bat0)-weight_order(o0)+weight_order(o1)
    w1=weight_batch(bat1)-weight_order(o1)+weight_order(o0)
    if (w0<=18 and w1<=18):
        return True
    return False

In [48]:
#calculate f(x')
#function dist_batches
def dist_bats(bats,i):
    d=0
    if i==0:
        for b in bats:
            d+=dist_tour(batchToTour(b,['OutD0','OutD0']))
    elif i==1:
        for b in bats:
            d+=dist_tour(batchToTour(b,['OutD1','OutD1']))
    return d

In [49]:
#Input batches_c0 and batches_c1 should be from shaking function!
batches_c0 = [['2', '0', '3'], ['1'],['6'], ['8']]
batches_c1 = [['4', '7'], ['5'], ['9']]

def localSearch(batches_c0,batches_c1):
    boo=True
    k=0
    while boo:
        k+=1
        print(k)
        # for 1 iteration
        f_prev=dist_bats(batches_c0,0)+dist_bats(batches_c1,1)
        kappa=0
        batches_c0_copy=batches_c0
        batches_c1_copy=batches_c1
        while (kappa <= 2):
            kappa+=1
            b0=random.choice(batches_c0_copy)
            b1=random.choice(batches_c1_copy)
            o0=random.choice(b0)
            o1=random.choice(b1)
            #check constraints
            if check_wrap(b0,b1,o0,o1):
                print(kappa,'True')
                #wrapping two orders
                idx0=batches_c0_copy.index(b0)
                idx1=batches_c1_copy.index(b1)
                b0[b0.index(o0)]=o1
                b1[b1.index(o1)]=o0
                batches_c0_copy[idx0]=b1
                batches_c1_copy[idx1]=b0
            #run greedy for new wrapped batches
            lst=[]
            for b in batches_c0:
                lst.append(batchToTour(b,['OutD0','OutD0']))
            for b in batches_c0:
                lst.append(batchToTour(b,['OutD1','OutD1']))
            #then run SA
            lst_SA=[]
            for tour in lst:
                lst_SA.append(SA(tour))
            #calculate f_next
            f_next=0
            for tour in lst_SA:
                f_next+=dist_tour(tour)
            if f_prev<f_next:
                #print('True')
                batches_c0=batches_c0_copy
                batches_c1=batches_c1_copy
            #print('output format will be=', batches_c0,batches_c1, lst_SA)
        if k==10:
            boo=False
    return batches_c0,batches_c1,lst_SA

In [50]:
batches_c0,batches_c1,lst_SA=localSearch(batches_c0,batches_c1)

1
1 True
3 True
2
2 True
3 True
3
1 True
2 True
3 True
4
1 True
2 True
5
2 True
3 True
6
7
2 True
3 True
8
1 True
2 True
3 True
9
1 True
2 True
3 True
10
2 True
3 True


In [51]:
batches_c0,batches_c1

([['3'], ['1'], ['2', '4', '9'], ['8']], [['7'], ['5'], ['0', '6']])